# mp / Telnet like client

In [ ]:
import socket, os, time

code = [
    "print('empty string follows')",
    "",
    "print(5**9)",
    "print(2**23)",
"""
uid = bytes(6)
try:
    import machine
    uid = machine.unique_id()
except:
    import microcontroller
    uid = microcontroller.cpu.uid
print(":".join("{:02x}".format(x) for x in uid), end="")
""",
"""
for i in range(5):
    print(i, i**5)
""",
"""
print('a nasty error')
5/0
""",
    "'back to nice'",
]

ADDR = ('10.39.40.129', 54321)

EOT = b'\x04'

class NetClient:
    
    def __init__(self, addr):
        self.addr = addr
        
    def recv_exactly(self, sz:int):
        # read sz bytes from socket
        data = b''
        while len(data) < sz:
            d = self.sock.recv(sz)
            if len(d) == 0:
                raise Exception("connection closed")
            data += d
        # print(f"_recv_exactly [{sz}] {data}")
        return data
    
    def readlineX(self):
        return self.sock.makefile().readline().strip()

    def readline(self, timeout=1):
        """Blocking, but interrupt with ."""
        res = b''
        start = time.monotonic()
        # let's give the MCU some time ...
        while (time.monotonic() - start) < timeout:
            try:
                b = self.sock.recv(1)
            except socket.timeout:
                time.sleep(0.1)
                continue
            if b == b'' or b == b'\n':
                return res.decode()
            res += b
        return res.decode()

    def connect(self):
        self.sock = s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect(self.addr)
        s.settimeout(1)
        print("connected")
        version = self.readline()
        assert version == VERSION, "Version mismatch"
        s.sendall(PWD.encode())
        s.sendall(b'\n')
        ok = self.readline()
        if ok != 'OK':
            raise Exception(ok)
        return self
    
    def __enter__(self):
        return self

    def __exit__(self, *args):
        self.sock.sendall(b'bye\n')
        try:
            self.sock.close()
        except:
            pass
        self.sock = None
        


        
with NetClient(ADDR) as client:
    client.connect()
    if True:
        for c in code:
            client.do_eval_exec(c)
    if True:
        client.do_echo(b'abc')
        client.do_echo(bytes(os.urandom(20000)))
        client.do_fput('airlift/libs/server/esp32/lib/urpc/server.py', 'dummy.txt')
        client.do_fget('dummy.txt', 'delete_me.txt')
    if True:
        with open('airlift/libs/server/esp32/lib/urpc/server.py') as f:
            c = f.read()
        code = c + c + c + '\nprint("Got it!")\n'
        print('len(code)', len(code))
        client.do_eval_exec(code)